 # 爬蟲 momo購物網 post 練習

In [11]:
import requests

URL = "http://www.momoshop.com.tw/search/"

headers = {'user-agent': 'Mozilla/5.0 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4)'
           ' AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 '
           'Safari/605.1.15'}

r = requests.get(URL +"searchShop.jsp?keyword=iPhone", headers=headers)

print(r.status_code)  # 200 means success

200


# PTT Soft_Job 文章

In [3]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.ptt.cc/bbs/Soft_Job/index.html"

r = requests.get(URL)
if r.status_code == requests.codes.ok: # 200
    r.encoding = "utf8"
    soup = BeautifulSoup(r.text, "lxml")
    tag_divs = soup.find_all("div", class_="r-ent")
    for tag in tag_divs:
        tag_a = tag.find("a")
        print(tag_a["href"])
        print(tag_a.text)
        print(tag.find("div", class_="author").string)
else:
    print("HTTP failed")

/bbs/Soft_Job/M.1554988690.A.903.html
[請益] 該如何證明自己演算法的能力？
iven00000000
/bbs/Soft_Job/M.1554989603.A.6B9.html
[請益] 筆記軟體推薦
PttZF
/bbs/Soft_Job/M.1554998165.A.8EB.html
[請益] 未來發展與薪水抉擇
kay0629
/bbs/Soft_Job/M.1501827536.A.DF2.html
[公告] 本板板規  2017/4/10更新
MOONY135
/bbs/Soft_Job/M.1501827692.A.18D.html
[公告] 徵才不符板規或徵才自刪公司
MOONY135
/bbs/Soft_Job/M.1501847358.A.F41.html
[情報] 訓練課程與付費APP與網站分享
MOONY135
/bbs/Soft_Job/M.1501847445.A.045.html
[情報] 社群活動與免費APP與網站分享
MOONY135
/bbs/Soft_Job/M.1498710085.A.78A.html
[板務] 請板友提供板務建議
s89227


# 爬股票資訊

In [17]:
import time
import requests
import csv
from bs4 import BeautifulSoup

#目標網址
URL = "https://tw.stock.yahoo.com/q/q?s=" #後面加要的股票數字代碼



def generate_urls(url, stocks):
    urls = []
    for stock in stocks:
        urls.append(url + stock)
    return urls

def web_scraping_bot(urls):
    stocks = [["代碼", "名稱", "狀態", "股價", "昨收", "張數", "最高", "最低"]]
    
    for url in urls:
        stock_id = url.split("=")[-1]
        print(stock_id)
        print('抓取： '+ stock_id + " 網路資料中.....")
        r = get_resource(url)
        if r.status_code == requests.codes.ok:
            soup = parse_html(r.text)
            stock = get_stock(soup, stock_id)
            stocks.append(stock)
            print("wait 5 second....")
            time.sleep(5)
        else:
            print("HTTP failed ...")
    return stocks

def get_resource(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4)'
               'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Safari/605.1.15'}
    return requests.get(url, headers=headers)

def parse_html(html_str):
    return BeautifulSoup(html_str, "lxml")

def get_stock(soup, stock_id):
    table = soup.find_all(text="成交")[0].parent.parent.parent
    status = table.select("tr")[0].select("th")[2].text
    name =   table.select("tr")[1].select("td")[0].text
    price =  table.select("tr")[1].select("td")[2].text
    yclose = table.select("tr")[1].select("td")[7].text
    volume = table.select("tr")[1].select("td")[6].text
    high =   table.select("tr")[1].select("td")[9].text
    low  =   table.select("tr")[1].select("td")[10].text
    return [stock_id, name[4:-6], status, price, yclose, volume, high, low]

def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf-8") as fp:
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)

def web_scraping_bot(urls):
    stocks = [["代碼","名稱","狀態","股價","昨收","張數","最高","最低"]]
    
    for url in urls:
        stock_id = url.split("=")[-1]
        print("抓取: " + stock_id + " 網路資料中...")
        r = get_resource(url)
        if r.status_code == requests.codes.ok:
            soup = parse_html(r.text)
            stock = get_stock(soup, stock_id)
            stocks.append(stock)
            print("等待5秒鐘...")
            time.sleep(5) 
        else:
            print("HTTP請求錯誤...")       

    return stocks

if __name__ == "__main__":
    urls = generate_urls(URL, ["3711", "2330", "2454"])
    # print(urls)
    stocks = web_scraping_bot(urls)
    for stock in stocks:
        print(stock)
    save_to_csv(stocks, "stocks.csv")

抓取: 3711 網路資料中...
等待5秒鐘...
抓取: 2330 網路資料中...
等待5秒鐘...
抓取: 2454 網路資料中...
等待5秒鐘...
['代碼', '名稱', '狀態', '股價', '昨收', '張數', '最高', '最低']
['3711', '日月光投控', '成交', '71.4', '71.8', '4,781', '71.8', '70.7']
['2330', '台積電', '成交', '252.0', '254.0', '24,870', '254.0', '251.5']
['2454', '聯發科', '成交', '291.5', '300.0', '12,161', '303.0', '291.0']


# 爬Yahoo本週電影新片

In [19]:
import time
import requests
import csv
from bs4 import BeautifulSoup
import re

#目標網址
URL = "https://movies.yahoo.com.tw/movie_thisweek.html?page={0}"

def generate_urls(url, start_page, end_page):
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(page))
    return urls

def get_resource(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4)'
               'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Safari/605.1.15'}
    return requests.get(url, headers=headers)

def parse_html(html_str):
    return BeautifulSoup(html_str, "lxml")

def format_date(date_str):
    # 取出上映日期
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)

def get_movies(soup):
    movies = []
    rows = soup.find_all("div", class_="release_info_text")
    for row in rows:
        movie_name_div = row.find("div", class_="release_movie_name")
        cht_name = movie_name_div.a.text.strip()
        eng_name = movie_name_div.find("div", class_="en").a.text.strip()
        expectation = row.find("div", class_="leveltext").span.text.strip()
        photo = row.parent.find_previous_sibling(
                "div", class_="release_foto")
        poster_url = photo.a.img["src"]
        release_date = format_date(row.find('div', 'release_movie_time').text)
        
        movie= [cht_name,eng_name,expectation,
                poster_url,release_date]
        movies.append(movie)
    return movies

def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf-8") as fp:
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)

def web_scraping_bot(urls):
    all_movies = [["中文片名","英文片名","期待度","海報圖片","上映日"]]
    page = 1
    
    for url in urls:
        print("抓取: 第" + str(page) + "頁 網路資料中...")
        page = page + 1
        r = get_resource(url)
        if r.status_code == requests.codes.ok:
            soup = parse_html(r.text)
            movies = get_movies(soup)
            all_movies = all_movies + movies
            print("等待5秒鐘...")
            if soup.find("li", class_="nexttxt disabled"):
                break   # 已經沒有下一頁
            time.sleep(5) 
        else:
            print("HTTP請求錯誤...")

    return all_movies

if __name__ == "__main__":
    urls = generate_urls(URL, 1, 5)
    # print(urls)
    movies = web_scraping_bot(urls)
    for movie in movies:
        print(movie)
    save_to_csv(movies, "movies.csv")

抓取: 第1頁 網路資料中...
等待5秒鐘...
抓取: 第2頁 網路資料中...
等待5秒鐘...
['中文片名', '英文片名', '期待度', '海報圖片', '上映日']
['地獄怪客：血后的崛起', 'Hellboy', '95%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/November2018/Zj24hpguXm3NITIBjEgI-840x1200.jpg', '2019-04-11']
['禁入墳場', 'Pet Sematary', '90%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/February2019/zkaR7xCgJ9CNoJfyLvMI-1012x1500.JPG', '2019-04-12']
['人盡皆知', 'Everybody Knows', '68%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/February2019/4FYsUy16Xf9B6Uonmo2r-1984x2835.JPG', '2019-04-12']
['女王的柯基', 'The Queen’s Corgi', '76%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/April2019/HrLGWCSZ005b67JzSM9N-2500x3360.png', '2019-04-12']
['十二個想死的少年', '12 Suicidal Teens', '71%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/March2019/fLfIHQJO2uHWa3tYckBE-2487x3543.jpg', '2019-04-12']
['電競殺手', 'Censor', '68%', 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/March2019/m0n84JkfjX6CpqydexAp-